In [2]:
import torch.nn as nn
import torch.functional as F
import torch
import torch.optim as optim

NameError: name '_C' is not defined

In [ ]:
# Taken from paper
batch_size = 64
hidden_size = 512
embedding_size = 512 #LSTM, Layers
lr = 0.001
skip_num = 0
code_vocab_size = len(100)
#code_max_len = max_code
comment_vocab_size = len(100)
#comment_max_len = max_comm # This and max_c

In [ ]:
# Query is set of hidden staes, key is our strings up to the point. 
def attention(query, key, value):
    stats = torch.inner(query, key)
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)

    p_attn = F.softmax(scores, dim = -1)

    return torch.matmul(p_attn, value), p_attn

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, code_vocab, comm_vocab):
        super(ActorNetwork, self).__init__()
        self.hidden_size = hidden_size
        # these are probalby btached, investigate later
        # Create embeeddings
        self.enc_embedding_seq = nn.Embedding(code_vocab, hidden_size)
        self.enc_embedding_struct = nn.Embedding(code_vocab, hidden_size)

        #encode with LSTM
        self.enc_LSTM_seq = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.enc_LSTM_struct = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        # END ENCODE

    def forward(self, seq, struct):
        '''
        Takes Sequential and Structural representation of code and creates two hidden layers and outputs out of them.
        '''
        e_seq = self.enc_embedding_seq(seq)
        e_struct = self.enc_embedding_struct(struct)
        out_seq, hide_seq = self.enc_LSTM_seq(e_seq)
        out_struct, hide_struct = self.enc_LSTM_struct(e_struct)

        return out_seq, hide_seq, out_struct, hide_struct


class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, code_vocab, comm_vocab):
        # START DCODE
        self.dec_embedding = nn.Embedding(comm_vocab, hidden_size)
        self.relu = nn.ReLU()
        self.dec_LSTM = nn.LSTM(hidden_size, hidden_size)
        self.pred = nn.Linear(hidden_size,  comm_vocab, 1) # Predict 
    def forward(self, input, hidden):
        out = self.dec_embedding(input)
        out = self.relu(out)
        out, hidden = self.dec_LSTM(out, hidden)
        out = self.pred(out)

        return out, hidden 
class HybridAttention(nn.Module):
    def __init__(self, hidden_size, code_vocab, comm_vocab):
        # START DCODE
        self.dec_embedding = nn.Embedding(comm_vocab, hidden_size)
        self.relu = nn.ReLU()
        self.dec_LSTM = nn.LSTM(hidden_size, hidden_size)
        self.pred = nn.Linear(hidden_size,  comm_vocab, 1) # Predict 
    def forward(self, input, hidden):
        out = self.dec_embedding(input)
        out = self.relu(out)
        out, hidden = self.dec_LSTM(out, hidden)
        out = self.pred(out)

        return out, hidden 
    

SyntaxError: expected ':' (1496273468.py, line 63)

In [ ]:
encoder = Encoder(0, hidden_size, code_vocab_size, comment_vocab_size)
decoder = Decoder(0, hidden_size, code_vocab_size, comment_vocab_size)
encoder_optimizer = optim.Adam(encoder.parameters(), lr = lr)
decoder_optimizer = optim.Adam(decoder.parameters(), lr = lr)